# Оценка балла общей облачности (БОО, total cloud cover, TCC)

### Загрузим и предобработаем фотографии

In [1]:
import cv2
import random
import torch
import numpy as np 
import torchvision 
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import datetime
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
import pickle
from tqdm import tqdm
from typing import Tuple, List, Type, Dict, Any
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

In [2]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
!mkdir train

In [4]:
!mkdir test

In [5]:
!wget https://www.dropbox.com/s/nyy2q2zb1bkw3mi/train.tar.gz

--2021-05-03 09:13:56--  https://www.dropbox.com/s/nyy2q2zb1bkw3mi/train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nyy2q2zb1bkw3mi/train.tar.gz [following]
--2021-05-03 09:13:56--  https://www.dropbox.com/s/raw/nyy2q2zb1bkw3mi/train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc833f887e306d4607dd5961e499.dl.dropboxusercontent.com/cd/0/inline/BNy2XJhXBDiZ7NqrIGk6ddnDl38gRNFeedSjyqHkU2fVa2f3kkjMrau1kXISZ0J9W9BioF-iaJ1SZuMNRfhqZ-TbHF6H8NZRIybr1Tm1Vc9EzCDpji2Fl3wqu4RtN-mya_pjiH8WcrhFJuDwAXHIJ1W8/file# [following]
--2021-05-03 09:13:57--  https://uc833f887e306d4607dd5961e499.dl.dropboxusercontent.com/cd/0/inline/BNy2XJhXBDiZ7NqrIGk6ddnDl38gRNFeedSjyqHkU2fVa2f3kkjMrau1kXISZ0J9W9BioF-iaJ1SZuMNRfhqZ-T

In [6]:
!wget https://www.dropbox.com/s/a9k38jd1keuwnnv/test.tar.gz

--2021-05-03 09:17:34--  https://www.dropbox.com/s/a9k38jd1keuwnnv/test.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/a9k38jd1keuwnnv/test.tar.gz [following]
--2021-05-03 09:17:34--  https://www.dropbox.com/s/raw/a9k38jd1keuwnnv/test.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4a768dee4de6ebc8dcebd96256.dl.dropboxusercontent.com/cd/0/inline/BNw_dJNDkT0AZJi8HB9ck6QZd_Ji7X4k5nVnXMEmHoiIS1-ApakEsWLNosyt1nJ0KZNerRAoN4RfG8eoKCwjr7JI6U2bp85l2D16bYkruOdn5K_PGfjOHi62vX9U7ztOASAJQCgMUcgdsKTneCAfL3TC/file# [following]
--2021-05-03 09:17:35--  https://uc4a768dee4de6ebc8dcebd96256.dl.dropboxusercontent.com/cd/0/inline/BNw_dJNDkT0AZJi8HB9ck6QZd_Ji7X4k5nVnXMEmHoiIS1-ApakEsWLNosyt1nJ0KZNerRAoN4RfG8eoKCwjr7JI6U

In [7]:
!mv /content/test.tar.gz /content/test
%cd /content/test
!tar xvzf /content/test/test.tar.gz
%cd ..

Выходные данные были обрезаны до нескольких последних строк (5000).
./AMK71/snapshots/snapshots-2018-07-13/9cf438b0-540e-4678-b616-976c9f21094d.jpg
./AMK71/snapshots/snapshots-2018-07-13/f3937a9b-3f9c-4d35-b8d8-2e7cfd170760.jpg
./AMK71/snapshots/snapshots-2018-07-13/e9544e47-f355-46e8-b8dd-b42176f6d8fa.jpg
./AMK71/snapshots/snapshots-2018-07-13/40c7ba2b-3788-4e65-b2fa-b4cb76bc89fb.jpg
./AMK71/snapshots/snapshots-2018-07-13/41c8d3da-7ab1-4c6b-9336-81b10fa7cbf0.jpg
./AMK71/snapshots/snapshots-2018-07-13/750b657e-634b-40be-b5be-a39a1eeffc35.jpg
./AMK71/snapshots/snapshots-2018-07-13/cb420bcd-6e20-4d5c-b260-17b728d81676.jpg
./AMK71/snapshots/snapshots-2018-07-13/cc510420-645a-40f0-b35f-269458a132d4.jpg
./AMK71/snapshots/snapshots-2018-07-13/5b361bb4-7c8c-4be5-96b0-1367db26ef89.jpg
./AMK71/snapshots/snapshots-2018-07-13/78d7db1a-18b4-4906-a93b-99af67fc22d6.jpg
./AMK71/snapshots/snapshots-2018-07-13/11bd126a-9a18-4cb6-888d-894cbc152ea3.jpg
./AMK71/snapshots/snapshots-2018-07-13/bc51cd24-a4ae

In [8]:
!mv /content/train.tar.gz /content/train
%cd /content/train
!tar xvzf /content/train/train.tar.gz
%cd ..

Выходные данные были обрезаны до нескольких последних строк (5000).
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T07-56-43devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T13-59-25devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T07-55-13devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T13-04-48devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T07-03-06devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T12-01-11devID2.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T03-59-45devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T09-56-26devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T12-03-41devID1.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T14-00-55devID2.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T12-03-11devID2.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T05-59-59devID2.jpg
./ANS31/snapshots/snapshots-2015-12-25/img-2015-12-25T10

In [9]:
# !cd /content/logs && rm -rf ./*

In [10]:
def get_transform(train):
    transformations = []
    transformations.append(torchvision.transforms.ToTensor())
    # if train:
    #     transformations.append(transforms.RandomHorizontalFlip(0.5))
    return torchvision.transforms.Compose(transformations)

In [11]:
#подавать в конструктор имя корневой папки(train/test)

class CloudDataset(Dataset):
    def __init__(self, root, transforms, n_classes):
        self.root = root
        self.transforms = transforms
        self.num_classes = n_classes
        with open(f"{self.root}/index.pkl", 'rb') as f:
            self.data_index = pickle.load(f)

    def __getitem__(self, idx):
        data_dict = self.data_index[idx]

        #prepare date to correct form to read image correctly
        date = data_dict['observations_dt']
        month = str(date.month)
        day = str(date.day) 

        #change the date format
        if date.month < 10:
            month = "0" + month
        if date.day < 10:
            day = "0" + day

        #prepare pathes to image and mask
        snapshots_dir = f"snapshots-{date.year}-{month}-{day}"
        if data_dict['mission'] == "AI49":
            snapshots_dir = f"ai49-snapshots-{date.year}-{month}-{day}"
        path_to_photo = f"{self.root}/{data_dict['mission']}/snapshots/{snapshots_dir}/{data_dict['jpg_filename']}"
        path_to_mask = f"{self.root}/{data_dict['mask_fname']}"

        #read photo and mask
        cloud_photo = cv2.imread(path_to_photo)
        mask = cv2.imread(path_to_mask)

        #add some useful information about photo to dict
        target = {}
        target["date"] = np.array([date.year, date.month, date.day])
        name = data_dict['jpg_filename']

        target["label"] = 10
        if 'observed_TCC' in data_dict:
           target["label"] = data_dict['observed_TCC']

        if self.transforms is not None:
            cloud_photo = self.transforms(cloud_photo)
            #здесь же аугментировать маску, если нужно
            for _,value in target.items():
                if value is not None:
                    value = torch.tensor(value)
        return cloud_photo, target, name

    def __len__(self):
        return len(self.data_index)

In [12]:
batch_size = 64

In [13]:
train_dataset_full = CloudDataset("/content/train", get_transform(train=True), 9)

train_size = int(0.8 * len(train_dataset_full))
test_size = len(train_dataset_full) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset_full, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [14]:
test_dataset = CloudDataset("/content/test", get_transform(train=False), 9)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [15]:
len(train_dataset)

73661

In [16]:
len(test_dataset)

27000

In [17]:
import torchvision.models as models

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Метод 1: Решим задачу классификации (данный метод закомментирован, так как второй метод, который учитывает отношение порядка между классами, показал себя лучше на тестовых данных)

In [ ]:
resnet = models.resnet18(pretrained=True)
resnet

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
dir(resnet)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_impl',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_make_layer',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_norm_layer',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_d

In [ ]:
resnet.fc = torch.nn.Sequential(nn.Linear(512, 256),
                                nn.ReLU(),
                                nn.Linear(256, 64),
                                nn.ReLU(),
                                nn.Linear(64, 9))

In [ ]:
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def train_model(model: torch.nn.Module, 
                train_loader: DataLoader,
                val_loader: DataLoader,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                max_epochs = 10):
    
    #region TENSORBOARD tutorial
    tb_writer = SummaryWriter(log_dir='./logs/run003/')
    #endregion ##############################

    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)

    best_val_loss = None
    best_epoch = None
    
    loss_history = []
    #endregion borrowed from HW03
    
    for epoch in tqdm(range(max_epochs)):
        #region borrowed from HW03
        print(f'Epoch {epoch}')
        train_epoch_loss_history = train_single_epoch(model, optimizer, loss_function, train_loader, tb_writer, epoch)
        loss_history = loss_history+train_epoch_loss_history

        val_metrics = validate_single_epoch(model, loss_function, val_loader)

        #endregion ##############################
        
        #region TENSORBOARD tutorial
        # tb_writer.add_scalar('train_loss', np.sum(train_epoch_loss_history)/len(train_loader.dataset), global_step=epoch)
        # tb_writer.add_scalar('val_loss', val_metrics['loss'], global_step=epoch)
        # tb_writer.add_scalar('val_accuracy', val_metrics['accuracy'], global_step=epoch)
        #endregion ##############################

        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')
         
        # if epoch - best_epoch > early_stopping_patience:
        #     print('Early stopping triggered')
        #     return
        # endregion ##############################
        torch.save(resnet.state_dict(), f"resnet_on_epoch_{epoch}")

In [ ]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader,
                       tb_writer: SummaryWriter,
                       epoch: int):
    
    model.train()
    loss_value = []
    for x, y_dict,_ in tqdm(data_loader):
        x = x.to(device)
        y = y_dict["label"]
        y = y.to(device=device, dtype=torch.int64)
        optimizer.zero_grad()
        output = model(x)
        #print(y)
        #print(output)
        loss = loss_function(output, y)
        loss_value.append(loss)
        loss.backward()
        optimizer.step()
    print(loss_value)
    # for tag, param in model.named_parameters():
    #     tb_writer.add_histogram('grad/%s'%tag, param.grad.data.cpu().numpy(), epoch)
    #     tb_writer.add_histogram('weight/%s' % tag, param.data.cpu().numpy(), epoch)

    return loss_value

In [ ]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    
    val_results = {'loss': 0, 'accuracy': 0}
    
    model.eval()
    for x, y_dict,_ in tqdm(data_loader):
        x = x.to(device)
        y = y_dict["label"].long()
        y = y.to(device=device, dtype=torch.int64)
        output = model(x)
        val_results['loss'] += loss_function(output, y)
        val_results['accuracy'] += torch.sum(torch.argmax(output, dim=1) == y).to(torch.float)
    val_results['loss'] /= len(data_loader.dataset)
    val_results['accuracy'] /= len(data_loader.dataset)
    return val_results

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /content/logs/run003

In [ ]:
train_model(resnet, 
            train_loader=train_loader, 
            val_loader=val_loader, 
            loss_function=torch.nn.CrossEntropyLoss(),
            lr_scheduler_params={'factor': 0.3, 'patience': 5},
            initial_lr=0.01,
            max_epochs=1)

  0%|          | 0/576 [00:00<?, ?it/s]

Epoch 0



100%|██████████| 576/576 [1:57:58<00:00, 12.29s/it]

  0%|          | 0/144 [00:00<?, ?it/s]

[tensor(2.2228, device='cuda:0', grad_fn=<NllLossBackward>), tensor(42.2312, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.2843, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.4692, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.8541, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.0158, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.0106, device='cuda:0', grad_fn=<NllLossBackward>), tensor(1.8975, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.2713, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.1126, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.0615, device='cuda:0', grad_fn=<NllLossBackward>), tensor(1.9884, device='cuda:0', grad_fn=<NllLossBackward>), tensor(1.9509, device='cuda:0', grad_fn=<NllLossBackward>), tensor(1.9850, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.1506, device='cuda:0', grad_fn=<NllLossBackward>), tensor(1.9884, device='cuda:0', grad_fn=<NllLossBackward>), tensor(2.0430, device='cuda:0', grad_f


100%|██████████| 144/144 [28:29<00:00, 11.87s/it]


Validation metrics: 
{'loss': tensor(0.0071, device='cuda:0', grad_fn=<DivBackward0>), 'accuracy': tensor(0.6313, device='cuda:0')}
Best model yet, saving


100%|██████████| 1/1 [2:26:46<00:00, 8806.54s/it]


In [ ]:
#  from tensorboard import notebook
# notebook.list()

In [ ]:
# notebook.display(port=6006, height=1000)

In [ ]:
def test_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader):
    output = torch.zeros(1, 9).to(device)
    image_names = []
    model.eval()
    for x, _, name in tqdm(data_loader):
        x = x.to(device)
        y = model(x)
        output = torch.cat((output, model(x)), dim=0)
        image_names.append(name)
        print(output.size())
    return output, image_names

In [ ]:
y_pred_test, names = test_model(resnet, test_loader)

  0%|          | 1/422 [00:04<29:39,  4.23s/it]

torch.Size([65, 9])


  0%|          | 2/422 [00:16<45:31,  6.50s/it]

torch.Size([129, 9])


  1%|          | 3/422 [00:27<56:35,  8.10s/it]

...

torch.Size([26881, 9])


100%|█████████▉| 421/422 [1:22:57<00:11, 11.85s/it]

torch.Size([26945, 9])


100%|██████████| 422/422 [1:23:09<00:00, 11.82s/it]

torch.Size([27001, 9])


In [ ]:
y_pred_test_labels = torch.argmax(y_pred_test, dim=1)

In [ ]:
y_pred_test_labels.size()

torch.Size([27001])

In [ ]:
y_pred_test_labels = y_pred_test_labels[1:]

In [ ]:
y_pred_test_labels

tensor([0, 8, 1,  ..., 6, 0, 8], device='cuda:0')

In [ ]:
result = y_pred_test_labels.cpu().tolist()

In [ ]:
print(result)

[0, 8, 1, 8, 8, 0, 8, 1, 8, 6, 8, 0, 4, 0, 0, 8, 8, 8, 8, 8, 0, 0, 2, 4, 8, 1, 2, 1, 0, 0, 0, 2, 8, 7, 0, 0, 0, 8, 2, 2, 0, 8, 0, 6, 6, 0, 0, 8, 0, 4, 6, 7, 8, 0, 8, 7, 6, 8, 8, 7, 8, 2, 1, 8, 3, 8, 0, 0, 0, 2, 1, 8, 0, 0, 0, 4, 8, 8, 0, 6, 2, 1, 6, 1, 8, 0, 8, 0, 4, 8, 8, 8, 6, 2, 0, 0, 3, 4, 8, 8, 0, 6, 8, 0, 7, 4, 6, 8, 2, 8, 0, 6, 0, 4, 6, 7, 0, 8, 0, 0, 1, 8, 6, 4, 0, 2, 0, 7, 8, 7, 3, 1, 8, 6, 0, 0, 1, 6, 8, 8, 8, 1, 0, 4, 1, 1, 8, 0, 2, 1, 0, 8, 8, 8, 8, 6, 0, 0, 0, 1, 1, 7, 0, 6, 8, 2, 2, 8, 2, 6, 4, 6, 4, 8, 8, 6, 4, 6, 0, 0, 0, 7, 0, 0, 0, 0, 0, 8, 1, 0, 8, 4, 6, 0, 0, 8, 8, 8, 6, 8, 8, 6, 0, 3, 4, 8, 4, 8, 6, 8, 0, 0, 6, 6, 4, 6, 8, 0, 1, 7, 8, 8, 0, 2, 0, 0, 1, 8, 0, 8, 6, 8, 1, 8, 8, 0, 0, 0, 0, 2, 2, 0, 7, 8, 4, 0, 4, 8, 0, 1, 0, 1, 1, 0, 2, 1, 1, 8, 0, 0, 7, 6, 6, 0, 8, 8, 4, 6, 1, 8, 8, 1, 4, 8, 7, 6, 7, 0, 4, 0, 0, 2, 4, 4, 0, 7, 8, 0, 8, 1, 0, 6, 7, 1, 7, 8, 0, 8, 6, 0, 8, 2, 0, 8, 1, 3, 6, 1, 0, 8, 3, 6, 7, 4, 0, 8, 1, 0, 0, 8, 2, 1, 7, 0, 6, 7, 8, 0, 8, 0, 0, 0, 7, 

In [ ]:
name_list = []
for tup in names:
    for name in tup:
        name_list.append(name)

In [ ]:
import pandas as pd


data = dict(jpg_filename=name_list, TCC=result)

df = pd.DataFrame(data)

df.to_csv(r'result_resnet.csv', sep=',', index=False)

## Метод 2: Multi target network

In [19]:
class Multi_target_Net(nn.Module):
    def __init__(self, network, input_dim, n_classes):
        super().__init__()
        self.basic_net = network
        self.num_classes = n_classes
        self.classifier = nn.Sequential(nn.Linear(input_dim, 128),
                                        nn.ReLU(),
                                        nn.Linear(128, 64),
                                        nn.ReLU(),
                                        nn.Linear(64, self.num_classes))
        
        self.regressor = nn.Sequential(nn.Linear(input_dim, 128),
                                        nn.ReLU(),
                                        nn.Linear(128, 64),
                                        nn.ReLU(),
                                        nn.Linear(64, 1))
    def forward(self, input_data):
        basic_output = self.basic_net(input_data)
        classifier_output = self.classifier(basic_output)
        regressor_output = self.regressor(basic_output)

        return classifier_output, regressor_output

In [20]:
def train_model_multi_target(model: torch.nn.Module, 
                train_loader: DataLoader,
                val_loader: DataLoader,
                loss_classifier: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                loss_regressor: torch.nn.Module = torch.nn.MSELoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                max_epochs = 10):

    optimizer = torch.optim.Adam([
                {'params': model.basic_net.parameters(), 'lr': 1e-4},
                {'params': model.classifier.parameters(), 'lr': initial_lr},
                {'params': model.regressor.parameters(), 'lr': initial_lr}
            ], **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)

    best_val_loss = None
    best_epoch = None
    
    loss_history = []
    
    for epoch in tqdm(range(max_epochs)):

        print(f'Epoch {epoch}')
        train_epoch_loss_history = train_single_epoch_multi_target(model, optimizer, loss_classifier, loss_regressor, train_loader, epoch)
        loss_history = loss_history+train_epoch_loss_history

        val_metrics = validate_single_epoch_multi_target(model, loss_classifier, loss_regressor, val_loader)

        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')

        torch.save(model.state_dict(), f"resnet_on_epoch_{epoch}.pth")

In [21]:
def train_single_epoch_multi_target(model: torch.nn.Module,
                                    optimizer: torch.optim.Optimizer, 
                                    loss_classifier: torch.nn.Module,
                                    loss_regressor: torch.nn.Module,
                                    data_loader: torch.utils.data.DataLoader,
                                    epoch: int):
    
    model.train()
    loss_value = []
    for x, y_dict,_ in tqdm(data_loader):
        x = x.to(device)
        y = y_dict["label"]
        y = y.to(device=device, dtype=torch.int64)
        optimizer.zero_grad()
        output_class, output_regr = model(x)

        loss = loss_classifier(output_class, y) + loss_regressor(output_regr, y.view(-1, 1).float())
        loss.backward()
        loss_value.append(loss.item())
        optimizer.step()
    print(loss_value)

    return loss_value

In [22]:
def validate_single_epoch_multi_target(model: torch.nn.Module,
                                       loss_classifier: torch.nn.Module, 
                                       loss_regressor: torch.nn.Module,
                                       data_loader: torch.utils.data.DataLoader):
    
    val_results = {'loss': 0, 'accuracy': 0}
    
    model.eval()
    with torch.no_grad():
        for x, y_dict,_ in tqdm(data_loader):
            x = x.to(device)
            y = y_dict["label"].long()
            y = y.to(device=device, dtype=torch.int64)
            output_class, output_regr = model(x)
            val_results['loss'] += (loss_classifier(output_class, y) + loss_regressor(output_regr, y.view(-1,1).float())).item()
            val_results['accuracy'] += torch.sum(torch.argmax(output_class, dim=1) == y).to(torch.float)
        val_results['loss'] /= len(data_loader.dataset)
        val_results['accuracy'] /= len(data_loader.dataset)
    return val_results

In [23]:
resnet_18 = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
# for param in resnet_18.parameters():
#     param.requires_grad = False

In [ ]:
# for param in resnet_18.layer4.parameters():
#     param.requires_grad = True
# for param in resnet_18.avgpool.parameters():
#     param.requires_grad = True

In [24]:
resnet_18.fc = nn.Linear(512, 256)

In [25]:
multi_target_resnet = Multi_target_Net(resnet_18, 256, 9)

#load model
multi_target_resnet.load_state_dict(torch.load("/content/resnet_on_epoch_0(3).pth")) #path 

<All keys matched successfully>

In [26]:
multi_target_resnet.to(device)

Multi_target_Net(
  (basic_net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [27]:
train_model_multi_target(multi_target_resnet, 
            train_loader=train_loader, 
            val_loader=val_loader, 
            loss_classifier=torch.nn.CrossEntropyLoss(),
            loss_regressor=torch.nn.MSELoss(),
            lr_scheduler_params={'factor': 0.3, 'patience': 5},
            initial_lr=0.001,
            max_epochs=1) #в итоге обучала 3 эпохи

  0%|          | 0/1151 [00:00<?, ?it/s]

Epoch 0



100%|██████████| 1151/1151 [1:02:33<00:00,  3.26s/it]

  0%|          | 0/288 [00:00<?, ?it/s]

[0.6663299798965454, 0.6708191633224487, 0.6343038082122803, 0.5463064312934875, 0.6744610071182251, 0.6296511888504028, 0.8128490447998047, 1.2589902877807617, 0.719236433506012, 0.48655250668525696, 0.6922545433044434, 0.6493518948554993, 0.6792365908622742, 0.5264095067977905, 0.6163634061813354, 0.5545308589935303, 0.4086681008338928, 0.8105446696281433, 0.9900749921798706, 0.755449652671814, 0.6107498407363892, 0.6432687044143677, 1.0874168872833252, 0.7116081118583679, 0.6366557478904724, 0.4736199378967285, 0.6883014440536499, 0.4294874668121338, 0.7131887674331665, 0.6543144583702087, 1.0692672729492188, 0.5253725647926331, 0.31409043073654175, 0.614658534526825, 0.6051855087280273, 0.5316298007965088, 0.8440932035446167, 0.5175631046295166, 0.4495544731616974, 0.7163444757461548, 0.4219556450843811, 0.7174240350723267, 0.49843499064445496, 0.5509249567985535, 0.4518362283706665, 0.7104607820510864, 0.6243650913238525, 0.5385876893997192, 0.4185120463371277, 0.5043749213218689,


100%|██████████| 288/288 [08:44<00:00,  1.82s/it]


Validation metrics: 
{'loss': 0.005772416360941687, 'accuracy': tensor(0.9039, device='cuda:0')}
Best model yet, saving


100%|██████████| 1/1 [1:11:18<00:00, 4278.54s/it]


In [28]:
def test_model_multi_target(model: torch.nn.Module, 
                            data_loader: torch.utils.data.DataLoader):
    output = torch.zeros(1, 9).to(device)
    image_names = []
    model.eval()
    with torch.no_grad():
        for x, _, name in tqdm(data_loader):
            x = x.to(device)
            y, _ = model(x)
            output = torch.cat((output, y), dim=0)
            image_names.append(name)
    return output, image_names

In [29]:
y_pred_test_multi_target, names = test_model_multi_target(multi_target_resnet, test_loader)

100%|██████████| 422/422 [12:09<00:00,  1.73s/it]


In [30]:
y_pred_test_multi_target_labels = torch.argmax(y_pred_test_multi_target, dim=1)
y_pred_test_multi_target_labels = y_pred_test_multi_target_labels[1:]

In [31]:
result_multi_target = y_pred_test_multi_target_labels.cpu().tolist()

In [32]:
name_list = []
for tup in names:
    for name in tup:
        name_list.append(name)

In [33]:
import pandas as pd
data_mt = dict(jpg_filename=name_list, TCC=result_multi_target)

df = pd.DataFrame(data_mt)

df.to_csv(r'result_resnet_multi_target.csv', sep=',', index=False)